In [1]:
import matplotlib.colors as mcolors
import numpy as np
import polyscope as ps
import random

In [2]:
# Initialize polyscope and set some scene options
ps.init()

ps.set_build_gui(False)
ps.set_ground_plane_mode("none")
ps.set_SSAA_factor(2)
ps.set_background_color([0, 0, 0])

image_res = 512
ps.set_window_resizable(True)
ps.set_window_size(image_res / 2, image_res / 2) 

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 INTEL-22.5.11


In [3]:
def draw_circle_helper(center_x, center_y, radius, color, id):
    # Generate mesh geometry
    n_faces = 100
    verts, faces = [], []
    z = id * -0.1   # Small offset to prevent shapes from clipping into each other

    verts.append([center_x, center_y, z])
    thetas = np.linspace(0, 2 * np.pi, num=n_faces + 1)
    for theta in thetas[:-1]:
        x = radius * np.cos(theta) + center_x
        y = radius * np.sin(theta) + center_y
        verts.append([x, y, z])
    verts = np.array(verts)

    for i in range(n_faces - 1):
        faces.append([0, i+1, i+2])
    faces.append([0, n_faces, 1])
    faces = np.array(faces)

    circle = ps.register_surface_mesh(f'circle {str(id)}', verts, faces, enabled=True)
    circle.set_color(color)
    return circle

def draw_square_helper(center_x, center_y, side, color, id):
    z = id * -0.1
    half = side / 2

    verts = np.array([
        [center_x - half, center_y + half, z],
        [center_x - half, center_y - half, z],
        [center_x + half, center_y - half, z],
        [center_x + half, center_y + half, z]
    ])

    faces = np.array([
        [0, 1, 2],
        [0, 2, 3]
    ])

    square = ps.register_surface_mesh(f'square {str(id)}', verts, faces, enabled=True)
    square.set_color(color)
    return square

In [4]:
def dead_leaves(sigma, color_palette, res=512, max_iters=50, shape_mode='mixed'):
    """
    https://github.com/mbaradad/learning_with_noise/blob/main/generate_datasets/dead_leaves/generate_dataset.py
    
    sigma           -- determines distribution of radiis
    color_palette   -- a numpy array of colors, of shape (n, 3) or (n, 4)
    res             -- image resolution
    max_iters       -- number of shapes to draw
    shape_mode      -- whether to draw circles, squares, or a mixture
    """
    r_min = 0.05
    r_max = 0.2

    # Compute distribution of radiis (exponential distribution with lambda = sigma)
    k = 200
    r_list = np.linspace(r_min, r_max, k)
    r_dist = 1.0 / (r_list ** sigma)
    if sigma > 0:
        # Normalize
        r_dist = r_dist - (1 / r_max ** sigma)
    r_dist = np.cumsum(r_dist)
    # Normalize so that cumulative sum is 1
    r_dist = r_dist / r_dist.max()

    available_shapes = ['circle', 'square']
    assert shape_mode in available_shapes or shape_mode == 'mixed'
    for i in range(max_iters):
        if shape_mode == 'mixed':
            shape = random.choice(available_shapes)
        else:
            shape = shape_mode
        
        # TODO: sample a color randomly from color_palette
        color_index = np.random.choice(color_palette.shape[0])
        color = color_palette[color_index]

        r_p = np.random.uniform(0, 1)
        r_i = np.argmin(np.abs(r_dist - r_p))
        radius = max(int(r_list[r_i] * res), 1)
        
        center_x, center_y = np.array(np.random.uniform(0, res, size=2), dtype='int32')
        if shape == 'circle':
            draw_circle_helper(center_x, center_y, radius, color, i)
        elif shape == 'square':
            side = radius * np.sqrt(2)
            draw_square_helper(center_x, center_y, side, color, i)
        else:
            raise Exception(f'Got unsupported shape mode {shape}')


In [5]:
# Generate some rainbow colors
hues = np.linspace(0, 0.9, 100)
color_palette = np.array([mcolors.hsv_to_rgb([hue, 1, 1]) for hue in hues])
dead_leaves(0.5, color_palette, shape_mode='mixed')

In [6]:
ps.show()